In [18]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("captainozlem/framingham-chd-preprocessed-data")

print("Path to dataset files:", path)

Path to dataset files: /home/morning/.cache/kagglehub/datasets/captainozlem/framingham-chd-preprocessed-data/versions/1


In [19]:
import os
os.listdir(path)

['CHD_preprocessed.csv']

In [20]:
import pandas as pd
import fairtl as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [21]:
df = pd.read_csv(f'{path}/CHD_preprocessed.csv')
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,1,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,1,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,1,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [22]:
df.shape

(4133, 16)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4133 entries, 0 to 4132
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4133 non-null   int64  
 1   age              4133 non-null   int64  
 2   education        4133 non-null   int64  
 3   currentSmoker    4133 non-null   int64  
 4   cigsPerDay       4133 non-null   float64
 5   BPMeds           4133 non-null   float64
 6   prevalentStroke  4133 non-null   int64  
 7   prevalentHyp     4133 non-null   int64  
 8   diabetes         4133 non-null   int64  
 9   totChol          4133 non-null   float64
 10  sysBP            4133 non-null   float64
 11  diaBP            4133 non-null   float64
 12  BMI              4133 non-null   float64
 13  heartRate        4133 non-null   float64
 14  glucose          4133 non-null   float64
 15  TenYearCHD       4133 non-null   int64  
dtypes: float64(8), int64(8)
memory usage: 516.8 KB


In [24]:
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,1,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,1,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,1,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [25]:
df['male'].unique()

array([1, 0])

In [26]:
y = df['TenYearCHD'].values
X = df.drop('TenYearCHD', axis=1)

In [27]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(4133, 15)
(4133, 12)


In [28]:
# build mask
gender_0_mask = df['male'] == 0
gender_1_mask = df['male'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  2367
Male:  1766


In [29]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=10, shuffle=True, random_state=seed)

In [30]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


...

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 9 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 10 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
    Fold   Group   SVM_TPR   SVM_TNR   SVM_FPR   SVM_FNR  SVM_TP  SVM_TN  \
0      1  Female  0.000000  1.000000  0.000000  1.000000       0     203   
1      2  Female  0.033333  1.000000  0.000000  0.966667       1     207   
2      3  Female  0.000000  1.000000  0.000000  1.000000       0     211   
3      4  Female  0.000000  0.995215  0.004785  1.000000       0     208   
4      5  Female  0.032258  1.000000  0.000000  0.967742       1     206   
5      6  Female  0.000000  1.000000  0.000000  1.000000       0     208   
6      7  Female  0.000000  1.000000  0.000000  1.000000       0     203   
7      8  Female  0.000000  1.000000  0.000000  1.000000       0     214   
8      

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [31]:
results_df = pd.concat(results_list, ignore_index=True)

In [32]:
result_path = './results/k49_result.xlsx'
results_df.to_excel(result_path, index=False)

In [33]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,0.000000,1.000000,0.000000,1.000000,0,203,0,34,...,5,34,0.264706,0.881773,0.118227,0.735294,9,179,24,25
1,2,Female,0.033333,1.000000,0.000000,0.966667,1,207,0,29,...,5,27,0.333333,0.913043,0.086957,0.666667,10,189,18,20
2,3,Female,0.000000,1.000000,0.000000,1.000000,0,211,0,26,...,1,25,0.269231,0.867299,0.132701,0.730769,7,183,28,19
3,4,Female,0.000000,0.995215,0.004785,1.000000,0,208,1,28,...,5,28,0.142857,0.913876,0.086124,0.857143,4,191,18,24
4,5,Female,0.032258,1.000000,0.000000,0.967742,1,206,0,30,...,4,28,0.419355,0.888350,0.111650,0.580645,13,183,23,18


In [34]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: MannwhitneyuResult(statistic=np.float64(13.5), pvalue=np.float64(0.005390931627916147))
SVM - FPR: MannwhitneyuResult(statistic=np.float64(30.0), pvalue=np.float64(0.08369666249056641))
SVM - FN/FP: MannwhitneyuResult(statistic=np.float64(59.0), pvalue=np.float64(0.5197370415546618))
DT -TPR: TtestResult(statistic=np.float64(-2.0763498820219923), pvalue=np.float64(0.05967819856550617), df=np.float64(12.176874981183758))
DT - FPR: TtestResult(statistic=np.float64(-5.342269226945673), pvalue=np.float64(4.452992391325002e-05), df=np.float64(18.0))
DT - FN/FP: TtestResult(statistic=np.float64(0.0791304584392389), pvalue=np.float64(0.9378019432703095), df=np.float64(18.0))
RF -TPR: TtestResult(statistic=np.float64(-4.1434165608307305), pvalue=np.float64(0.0006103428850629118), df=np.float64(18.0))
RF - FPR: TtestResult(statistic=np.float64(-2.60572377606014), pvalue=np.float64(0.017882814571932353), df=np.float64(18.0))
RF - FN/FP: MannwhitneyuResult(statistic=np.float64(71.5), pv